In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import cv2
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Load the VGG16 model pre-trained on ImageNet
base_model = VGG16(weights='imagenet', include_top=False)

# Add new layers for classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

# Define the model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

def preprocess_image(image_path, target_size=(224, 224)):
    """
    Preprocess the image for model prediction.
    Args:
        image_path (str): Path to the image file.
        target_size (tuple): Desired size of the image.
    Returns:
        np.array: Preprocessed image.
    """
    image = load_img(image_path, target_size=target_size)
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    image = preprocess_input(image)
    return image

def calculate_vfi(prediction_score):
    """
    Calculate the Visual Field Index (VFI) based on the prediction score.
    Args:
        prediction_score (float): Prediction score from the model.
    Returns:
        float: Calculated VFI.
    """
    # Assuming VFI is inversely related to the prediction score
    vfi = (1 - prediction_score) * 100
    return vfi

def is_potentially_glaucomic(image_path, model):
    """
    This function uses a pre-trained CNN to predict if an image is potentially glaucomic.
    Args:
        image_path (str): Path to the image file.
        model (Model): Trained Keras model.
    Returns:
        bool, float: True if the model predicts glaucoma, False otherwise, and the calculated VFI.
    """
    # Preprocess the image
    image = preprocess_image(image_path)

    # Make prediction
    prediction = model.predict(image)[0][0]

    # Calculate VFI
    vfi = calculate_vfi(prediction)

    # Print analysis result
    is_glaucoma_suspected = prediction > 0.34
    status = "Glaucomic" if is_glaucoma_suspected else "Not Glaucomic"
    print(f"Image '{image_path}': Prediction score: {prediction:.2f}, Status: {status}, VFI: {vfi:.2f}")

    return is_glaucoma_suspected, vfi

def process_images(image_paths, model):
    """
    Process a list of images and predict potential glaucoma.
    Args:
        image_paths (list): List of image paths.
        model (Model): Trained Keras model.
    """
    for image_path in image_paths:
        is_potentially_glaucomic(image_path, model)

def load_image_paths(directory_path):
    """
    Load image paths from a directory.
    Args:
        directory_path (str): Path to the directory containing images.
    Returns:
        list: List of image paths.
    """
    image_paths = []
    for filename in os.listdir(directory_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_paths.append(os.path.join(directory_path, filename))
    return image_paths

def split_dataset(image_paths, train_size=0.7, validation_size=0.15, test_size=0.15):
    """
    Split the dataset into training, validation, and testing sets.
    Args:
        image_paths (list): List of image paths.
        train_size (float): Proportion of the dataset to include in the training set.
        validation_size (float): Proportion of the dataset to include in the validation set.
        test_size (float): Proportion of the dataset to include in the test set.
    Returns:
        tuple: Lists of image paths for training, validation, and testing sets.
    """
    train_paths, temp_paths = train_test_split(image_paths, train_size=train_size, random_state=42)
    val_paths, test_paths = train_test_split(temp_paths, test_size=test_size / (test_size + validation_size), random_state=42)
    return train_paths, val_paths, test_paths

# Example usage (replace with your image directory path)
directory_path = "/content/drive/MyDrive/tanishque_project/OS_folder"
image_paths = load_image_paths(directory_path)
train_paths, val_paths, test_paths = split_dataset(image_paths, train_size=0.7, validation_size=0.15, test_size=0.15)

print(f"Training set: {len(train_paths)} images")
print(f"Validation set: {len(val_paths)} images")
print(f"Testing set: {len(test_paths)} images")

# Process images in each set
print("\nProcessing training set...")
process_images(train_paths, model)

print("\nProcessing validation set...")
process_images(val_paths, model)

print("\nProcessing testing set...")
process_images(test_paths, model)

# Save the trained model
model.save("glaucoma_detection_model.h5")
print("Model saved to 'glaucoma_detection_model.h5'")

# Later, to load the model and use it for predictions:
loaded_model = load_model("glaucoma_detection_model.h5")
print("Model loaded from 'glaucoma_detection_model.h5'")

# Example of using the loaded model for prediction
example_image_path = "/content/XYX.png"
is_potentially_glaucomic(example_image_path, loaded_model)


In [5]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 857.8/857.8 kB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 10.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.1/318.1 kB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 54.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 8.2 M

In [7]:
def preprocess_image(image):
    """
    Preprocess the image for model prediction.
    Args:
        image (PIL.Image): Input image.
    Returns:
        np.array: Preprocessed image.
    """
    image = image.resize((224, 224))
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    image = preprocess_input(image)
    return image

def calculate_vfi(prediction_score):
    """
    Calculate the Visual Field Index (VFI) based on the prediction score.
    Args:
        prediction_score (float): Prediction score from the model.
    Returns:
        float: Calculated VFI.
    """
    # Assuming VFI is inversely related to the prediction score
    vfi = (1 - prediction_score) * 100
    return vfi

def predict_glaucoma(image):
    """
    Predict if an image is potentially glaucomic.
    Args:
        image (PIL.Image): Input image.
    Returns:
        str: Prediction result.
    """
    # Preprocess the image
    image = preprocess_image(image)

    # Make prediction
    prediction = model.predict(image)[0][0]

    # Calculate VFI
    vfi = calculate_vfi(prediction)

    # Determine if the image is glaucomic
    is_glaucoma_suspected = prediction > 0.34
    status = "Glaucomic" if is_glaucoma_suspected else "Not Glaucomic"

    return f"Prediction: {status}\nPrediction Score: {prediction:.2f}\nVFI: {vfi:.2f}"

# Create Gradio interface
interface = gr.Interface(
    fn=predict_glaucoma,
    inputs=gr.Image(type="pil"),
    outputs=gr.Textbox(),
    title="Glaucoma Detection",
    description="Upload an eye image to predict if it is potentially glaucomic."
)

# Launch the app
interface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://4e33a7612366b8c1f7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
